In [24]:
import advent
import numpy as np

In [25]:
# Basically my dumb version of an enum
pieces = ['D', 'T', 'J', 'I', 'O']

# convenience function for moving x/y coord:
tadd = lambda l, r: (l[0] + r[0], l[1] + r[1])

def highest_rock(board):
    # New rocks should spawn at y = highest_rock + 4
    if np.sum(board) == 0: return -1
    last_occurence = np.argmax((board.sum(axis=0)>0)[::-1])
    return board.shape[1] - last_occurence - 1

def start_position(piece, board):
    # The 'start' and the 'anchor' position is defined as the BOTTOM-LEFT piece
    # in case of the T-piece, this anchor will be empty! (but still the anchor)
    return (2, highest_rock(board) + 4)

def get_coordinates(piece, anchor):
    if piece == 'D': return [tadd(anchor, (0, 0)), tadd(anchor, (1, 0)), tadd(anchor, (2, 0)), tadd(anchor, (3, 0))]
    if piece == 'T': return [tadd(anchor, (1, 0)), tadd(anchor, (1, 1)), tadd(anchor, (1, 2)), tadd(anchor, (0, 1)), tadd(anchor, (2, 1))]
    if piece == 'J': return [tadd(anchor, (0, 0)), tadd(anchor, (1, 0)), tadd(anchor, (2, 0)), tadd(anchor, (2, 1)), tadd(anchor, (2, 2))]
    if piece == 'I': return [tadd(anchor, (0, 0)), tadd(anchor, (0, 1)), tadd(anchor, (0, 2)), tadd(anchor, (0, 3))]
    if piece == 'O': return [tadd(anchor, (0, 0)), tadd(anchor, (0, 1)), tadd(anchor, (1, 0)), tadd(anchor, (1, 1))]

def oob(coord):
    return coord[0] < 0 or coord[0] >= 7 or coord[1] < 0

def clear(board, piece, anchor):
    coordinates = get_coordinates(piece, anchor)
    if any(oob(coord) for coord in coordinates): return False
    if any(board[pixel] for pixel in coordinates): return False
    return True

def fall(board, piece, anchor):
    # Return boolean: if the fall was succesful
    # does not check if the piece was already clear, only if the new spaces are clear
    new_anchor = (anchor[0], anchor[1] - 1)
    if clear(board, piece, new_anchor): return new_anchor
    return None

def gust(board, direction, piece, anchor):
    # Returns true if gust is 'succesfull'/unblocked. direction = True if right, False if left
    if direction == '>': new_anchor = (anchor[0] + 1, anchor[1])
    elif direction == '<': new_anchor = (anchor[0] - 1, anchor[1])

    if clear(board, piece, new_anchor): return new_anchor
    return None
    
def save(board, piece, anchor):
    # saves in place!!!!
    coordinates = get_coordinates(piece, anchor)
    for pixel in coordinates:
        #print(f"saving {pixel}")
        board[pixel] = 1
    return board
    

In [26]:
def full_fall(board, piece, gust_data, gust_position):

    anchor = start_position(piece, board)
    # First gust, then fall, until the fall returns False
    while True:
        direction = gust_data[gust_position]
        new_anchor = gust(board, direction, piece, anchor)
        if new_anchor: anchor = new_anchor
        gust_position = (gust_position + 1) % len(gust_data)

        new_anchor = fall(board, piece, anchor)
        if new_anchor: anchor = new_anchor
        else: break

    save(board, piece, anchor)
    return (board, gust_position)


In [27]:
next_piece = {'D': 'T', 'T': 'J', 'J': 'I', 'I': 'O', 'O': 'D'}

def full_fall_n_times(board, gust_data, n=2022, piece='D', gust_position=0):
    for _ in range(n):
        board, gust_position = full_fall(board, piece, gust_data, gust_position)
        piece = next_piece[piece]
    return board, piece, gust_position

data = advent.get_lines('17')[0]
data = [c for c in data]

board = np.zeros((7, 5000))

board, p, _ = full_fall_n_times(board, data, n=2022)

print(highest_rock(board) + 1, p)
# 3194 is too high? but it works on the example input :(


3135 J


In [28]:
# Part 2: 1_000_000_000_000 turns

print(len(data))

10091
